In [ ]:
!pip install mendelai-brat-parser

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
import zipfile
import glob

import regex as re

from nltk.stem import WordNetLemmatizer
import spacy

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
# Import the claims positions and articles (claim-positions.txt).
print("\nImport claim-positions.txt:\n")
uploaded = files.upload()

# Import the premise positions and articles (premise-positions.txt).
print("\nImport premise-positions.txt:\n")
uploaded = files.upload()

# Import the POS tag to use in the POS BOW (original_POS.txt).
print("\nImport original_POS.txt:\n")
uploaded = files.upload()

# Import the simple POS tag to use in the simple POS BOW (original_simple_POS.txt).
print("\nImport original_simple_POS.txt:\n")
uploaded = files.upload()

# Import the POS tags bigrams to consider in the POS bigrams BOW (50_bigram_POS.txt).
print("\nImport 50_bigram_POS.txt:\n")
uploaded = files.upload()

# Import the simple POS tags bigrams to consider in the simple POS bigrams BOW (50_bigram_simple_POS.txt).
print("\nImport 50_bigram_simple_POS.txt:\n")
uploaded = files.upload()

# Import the dependency tags to consider in the dependency tag BOW(dependency_tag.txt).
print("\nImport dependency_tag.txt:\n")
uploaded = files.upload()

# Import the labeled named entity to consider in the dependency tag BOW(dependency_tag.txt).
print("\nImport labeled_named_entity.txt:\n")
uploaded = files.upload()

# Import the words to consider in the BOW (50_BOW.txt).
#print("\nImport 50_BOW.txt:\n")
#uploaded = files.upload()

# Import the bigrams to consider in the bigrams BOW (50_Bigram.txt).
#print("\nImport 50_Bigram.txt:\n")
#uploaded = files.upload()

# Import the text file previously produced containing the prediction scores
# of the subjectivity of each sentence in the dataset (subj_obj_Debater.txt).
print("\nImport subj_obj_Debater.txt:\n")
subjectivity_uploaded = files.upload()

# Import the dataframe previously produced containing the BOW on the dataset of 
# the n-grams indicated in the appendix of the guidelines of the Stab and Gurevych 
# dataset (2017 version) as indicative of an argumentative sections (claim or premise).
#(file: Debater_BOW_appendix_words.cvs)
print('\nImport Debater_BOW_appendix_words.cvs\n')
upload = files.upload()


Import claim-positions.txt:




Import premise-positions.txt:




Import original_POS.txt:




Import original_simple_POS.txt:




Import 50_bigram_POS.txt:




Import 50_bigram_simple_POS.txt:




Import dependency_tag.txt:




Import labeled_named_entity.txt:




Import subj_obj_Debater.txt:




Import Debater_BOW_appendix_words.cvs



In [ ]:
# Import the IBM Debater dataset.

uploaded = files.upload()

Saving IBM_Debater_(R)_CE-EMNLP-2015.v3.zip to IBM_Debater_(R)_CE-EMNLP-2015.v3.zip


In [ ]:
# Extract a zip file.
def import_zip(source: str, destination =""):
  with zipfile.ZipFile(source, 'r') as zip_ref:
    zip_ref.extractall(destination)

In [ ]:
# Extract the dataset.
import_zip("IBM_Debater_(R)_CE-EMNLP-2015.v3.zip")

# **Claim and premise extraction**

In [ ]:
# Read the files containing the labeled claim and premises.
claim_data=open("claim-positions.txt").read()
premise_data=open("premise-positions.txt").read()

In [ ]:
# Create a list which contains as elements the lines in the original document.
claim_lines=claim_data.split("\n")
premise_lines=premise_data.split("\n")

In [ ]:
# Separate each line of the document passed at the character '\t'.
# This character is the one used to separate the various component in each line
# of the document.
# In case of claims, also separate the first line from the rest of the document.
# The first line contains the description of the content of the other lines.

claim_list=[line.split("\t") for line in claim_lines][:-1]
premise_list=[line.split("\t") for line in premise_lines][:-1]

In [ ]:
claim_list[:2]

[['Sentence',
  'Article position in list',
  'Sentence starting point',
  'Sentence ending point'],
 ['exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world',
  '0',
  '418',
  '568']]

In [ ]:
# Create the two dataframes (trainsform the string representing numeric values into integer).

claim_dataframe=pd.DataFrame(claim_list[1:],columns=['sentence','article_id','start','end'])
claim_dataframe=claim_dataframe.astype({'article_id':int,'start':int,'end':int})

premise_dataframe=pd.DataFrame(premise_list[1:],columns=['sentence','article_id','start','end'])
premise_dataframe=premise_dataframe.astype({'article_id':int,'start':int,'end':int})

In [ ]:
claim_dataframe.head()

,sentence,article_id,start,end
0,exposure to violent video games causes at leas...,0,418,568
1,video game violence is not related to serious ...,0,829,907
2,some violent video games may actually have a p...,0,1004,1082
3,exposure to violent video games causes both sh...,0,1442,1577
4,they increase the violent tendencies among youth,0,3900,3948


In [ ]:
# Add the type of argumentative sections into the dataframes.
premise_type=["premise" for p in range(len(premise_list)-1)]
claim_type=["claim" for c in range(len(claim_list)-1)]

claim_dataframe["type"]=claim_type
premise_dataframe["type"]=premise_type

In [ ]:
claim_dataframe.head()

,sentence,article_id,start,end,type
0,exposure to violent video games causes at leas...,0,418,568,claim
1,video game violence is not related to serious ...,0,829,907,claim
2,some violent video games may actually have a p...,0,1004,1082,claim
3,exposure to violent video games causes both sh...,0,1442,1577,claim
4,they increase the violent tendencies among youth,0,3900,3948,claim


In [ ]:
# Concatenate the two dataframes into one.
argumentative_dataframe=pd.concat([claim_dataframe,premise_dataframe])

# **IOB and word list**

In [ ]:
# Extract original articles text.
import_zip("IBM_Debater_(R)_CE-EMNLP-2015.v3/articles.zip","IBM_Debater_(R)_CE-EMNLP-2015.v3")

In [ ]:
# Read the list of articles used in the dataset.
txt_articles = sorted(glob.glob("IBM_Debater_(R)_CE-EMNLP-2015.v3/articles/clean_*.txt"))

articles_text=[open(file).read() for file in txt_articles]

In [ ]:
articles_text[0]

'. \n\nControversies over video games often center on topics such as video game graphic violence, sex and sexism, violent and gory scenes, partial or full nudity, portrayal of criminal behavior, racism, and other provocative and objectionable material. \n\nVideo games have been studied for links to addiction and aggression. Earlier meta-analyses (an analysis of several studies) were conflicting. A 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world. A decrease in prosocial behavior (caring about the welfare and rights of others) was also noted [REF]. Another 2001 meta-analyses using similar methods[REF] and a more recent 2009 study focusing specifically on serious aggressive behavior[REF] concluded that video game violence is not related to serious aggressive behavior in real life. \n\nMany potential positive effects have been proposed [REF]. Recent research has sugg

In [ ]:
# Get the sorted list of the indicators of the articles used. 
articles_used=sorted(list(set(argumentative_dataframe['article_id'])))

In [ ]:
len(articles_used)

513

In [ ]:
articles_used[:5]

[0, 3, 19, 21, 32]

In [ ]:
# Get the start and finish points of each argumentative section of each of the text.
# Also get the type of each argumentative section.
# This points are indicated in the "start" and "end" coloumns of the Dataframe 
# created containing both types of argumentative sentences (argumentative_dataframe).

sorted_span=[sorted(list(argumentative_dataframe.loc[argumentative_dataframe['article_id'] == article,['start','end','type']].values), key=lambda element: element[0] ) for article in articles_used]

In [ ]:
len(sorted_span)

513

In [ ]:
sorted_span[0][0:15]

[array([394, 568, 'premise'], dtype=object),
 array([394, 670, 'premise'], dtype=object),
 array([394, 568, 'premise'], dtype=object),
 array([394, 670, 'premise'], dtype=object),
 array([394, 670, 'premise'], dtype=object),
 array([394, 568, 'premise'], dtype=object),
 array([394, 568, 'premise'], dtype=object),
 array([394, 568, 'premise'], dtype=object),
 array([394, 568, 'premise'], dtype=object),
 array([418, 568, 'claim'], dtype=object),
 array([673, 907, 'premise'], dtype=object),
 array([673, 907, 'premise'], dtype=object),
 array([673, 907, 'premise'], dtype=object),
 array([731, 907, 'premise'], dtype=object),
 array([731, 907, 'premise'], dtype=object)]

In [ ]:
# Join toghether all the argumentative section relating to the same section of 
# the text.

non_repeated_argumentative=[]

for article in sorted_span:
  non_repeated=[]
  start=article[0][0]
  end=article[0][1]
  argument_type=article[0][2]
  for i in range(1,len(article)):
    if end>article[i][0]:
      start=min(start,article[i][0])
      end=max(end,article[i][1])
      if not argument_type==article[i][2]:
        argument_type="both"
    else:
      non_repeated.append([start,end,argument_type])
      start=article[i][0]
      end=article[i][1]
      argument_type=article[i][2]
  non_repeated_argumentative.append(non_repeated)

In [ ]:
non_repeated_argumentative[0][0:10]

[[394, 670, 'both'],
 [673, 907, 'both'],
 [911, 1111, 'both'],
 [1323, 1577, 'both'],
 [2944, 3540, 'premise'],
 [3900, 3948, 'claim'],
 [3965, 4215, 'both'],
 [4217, 4479, 'premise'],
 [5514, 5803, 'both'],
 [6679, 7317, 'both']]

In [ ]:
# Number of argumentative section remaining.
sum([len(non_repeated) for non_repeated in non_repeated_argumentative])

2925

In [ ]:
# Tranform the couples of points ((start,end) of each non-repeated section) 
# to a list (separator of different section).
non_repeated_span_points=[]

for i in range(len(non_repeated_argumentative)):
  list_points=[]
  for sect in non_repeated_argumentative[i]:
    list_points.append(sect[0])
    list_points.append(sect[1])
  # Insert starting point of the text
  list_points.insert(0,0)
  list_points.append(len(articles_text[articles_used[i]]))
  non_repeated_span_points.append(list_points)

In [ ]:
non_repeated_span_points[0][0:10]

[0, 394, 670, 673, 907, 911, 1111, 1323, 1577, 2944]

In [ ]:
# Check if there are two argumentative section that are not separated by at least 
# one character in a non argumentative section.
# There could be more than one zero at the start in case the article starts
# with an argumentative sentences, there are no problems in that case.
argumentative_regions_connected=False

for i in range(len(non_repeated_span_points)):
  for j in range(len(non_repeated_span_points[i])-2):
      if (non_repeated_span_points[i][j]==non_repeated_span_points[i][j+1]) and (not j==0) :
        argumentative_regions_connected=True
        print(i)
        print(j)
print(argumentative_regions_connected)        

False


In [ ]:
# Separate the texts at the point indicated for the different sections

split_text=[]

for z in range(len(articles_used)):
  split_text.append([articles_text[articles_used[z]][i: j] for i, j in zip(non_repeated_span_points[z], non_repeated_span_points[z][1:])])

In [ ]:
split_text[0][:10]

['. \n\nControversies over video games often center on topics such as video game graphic violence, sex and sexism, violent and gory scenes, partial or full nudity, portrayal of criminal behavior, racism, and other provocative and objectionable material. \n\nVideo games have been studied for links to addiction and aggression. Earlier meta-analyses (an analysis of several studies) were conflicting. ',
 'A 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world. A decrease in prosocial behavior (caring about the welfare and rights of others) was also noted [REF',
 ']. ',
 'Another 2001 meta-analyses using similar methods[REF] and a more recent 2009 study focusing specifically on serious aggressive behavior[REF] concluded that video game violence is not related to serious aggressive behavior in real life',
 '. \n\n',
 'Many potential positive effects have been proposed [REF]

In [ ]:
len(split_text)

513

In [ ]:
# Remove articles with less than 6 section between argumentative and non argumentative.
not_remove=[i for i in range(len(split_text)) if len(split_text[i])>5]
split_text=[split_text[i] for i in not_remove]
non_repeated_argumentative=[non_repeated_argumentative[i] for i in not_remove]
articles_used=[articles_used[i] for i in not_remove]

In [ ]:
len(split_text)

271

In [ ]:
Y_IOB=[]
X_essay_word_list=[]

for i in range(len(split_text)):
  
  IOB=[]
  essay_word_list=[]

  # next_type is the next position still not considered in the list of argumentative sections of the essay examined.
  next_type=0

  texts=[re.sub(r"\n+"," ù ",text) for text in split_text[i]]
  # Remove the "[REF]" and "|-" from the text.
  texts=[re.sub(r"\[REF[\]$]*","",text) for text in texts]
  texts=[re.sub(r"^\]","",text) for text in texts]
  texts=[re.sub(r"|-","",text) for text in texts]

  # Set boolean value next_token_is_argumentative to false.
  # (first section is never argumentative, in case first sentence was argumentative
  # a fake section as been added previously).
  next_token_is_argumentative=False

  for section in texts:

    # Divide the text into token.
    seq=nltk.word_tokenize(section.lower())

    essay_word_list+=seq

    if next_token_is_argumentative:
      if non_repeated_argumentative[i][next_type][2]=='premise':
        IOB+=['B-P']
        for token in range(len(seq)-1):
          IOB+=['I-P']
      else:
        if non_repeated_argumentative[i][next_type][2]=='claim':
          IOB+=['B-C']
          for token in range(len(seq)-1):
            IOB+=['I-C']
        else:
          IOB+=['B-B']
          for token in range(len(seq)-1):
            IOB+=['I-B']
      next_type += 1
    else:
      for token in range(len(seq)):
        IOB+=['O']

    # An argumentative section is followed by a non-argumentative section and vice-versa    
    next_token_is_argumentative=not next_token_is_argumentative

  Y_IOB.append(IOB)
  X_essay_word_list.append(essay_word_list)

In [ ]:
for i in range(80,100):
  print(X_essay_word_list[0][i]+" - "+Y_IOB[0][i])

causes - I-B
at - I-B
least - I-B
a - I-B
temporary - I-B
increase - I-B
in - I-B
aggression - I-B
and - I-B
that - I-B
this - I-B
exposure - I-B
correlates - I-B
with - I-B
aggression - I-B
in - I-B
the - I-B
real - I-B
world - I-B
. - I-B


# **Articles' sentences list and statistic**

In [ ]:
# Get the list of sententences of each artile.

essays_sentence_list=[]

for essay_id in range(len(X_essay_word_list)):
  temp_essay_sentence_list=[]
  sentence=""

  for word_id in range(len(X_essay_word_list[essay_id])):
    
    word=X_essay_word_list[essay_id][word_id]

    # Add to the sentence every word that is not a simple \n or the end of the sentence (".").
    if not ( word=="ù" or word=="." ):
      sentence+=word+" "
          
    # . is the end of the sentence.       
    if word in [".","?","!"] and (not sentence==""):
      temp_essay_sentence_list.append(sentence)
      sentence=""
      
  essays_sentence_list.append(temp_essay_sentence_list)

In [ ]:
essays_sentence_list[0][0:2]

['controversies over video games often center on topics such as video game graphic violence , sex and sexism , violent and gory scenes , partial or full nudity , portrayal of criminal behavior , racism , and other provocative and objectionable material ',
 'video games have been studied for links to addiction and aggression ']

In [ ]:
sentences_word_list=[[nltk.word_tokenize(sentence) for sentence in essay] for essay in essays_sentence_list]

In [ ]:
sentences_word_list[0][0][:10]

['controversies',
 'over',
 'video',
 'games',
 'often',
 'center',
 'on',
 'topics',
 'such',
 'as']

In [ ]:
# Remove articles with sections of more than 150 words.
not_remove=[i for i in range(len(sentences_word_list)) if max([len(sentence) for sentence in sentences_word_list[i]])<=150]
X_essay_word_list=[X_essay_word_list[i] for i in not_remove]
Y_IOB=[Y_IOB[i] for i in not_remove]
split_text=[split_text[i] for i in not_remove]
non_repeated_argumentative=[non_repeated_argumentative[i] for i in not_remove]
articles_used=[articles_used[i] for i in not_remove]

In [ ]:
len(not_remove)

251

In [ ]:
# Get the list of all the sentences and the fact that they are part or contain 
# an argumentative section or not.

sentence_list=[]
sentences_argumentative_map=[]

# Also store the number of the paragraph containing the sentence in the essay,
# toghether with the order of the sentence in the paragraph and in the entire text. 
number_of_paragraph=[]
number_sentence_in_paragraph=[]
number_sentence_in_the_essay=[]
# List containing the id of the essay for each sentence in the corpus.
sentence_essay_id_list=[]
# Also store the sentence length.
sentence_number_words=[]

# Tokens to not consider as words in the words count.
ignore_tokens = ['\\', '~', '(', ')', '[', ']', '{', '}', '/', '"', "'", '^' ,'?', '!', '*', ',', '.', ';', ':']    

for essay_id in range(len(X_essay_word_list)):
  sentence=""
  argumentative=False
  claim=False
  premise=False
  num_paragraph=0
  num_in_paragraph=0
  num_in_essay=0
  sentence_length=0

  for word_id in range(len(X_essay_word_list[essay_id])):
    
    word=X_essay_word_list[essay_id][word_id]
    bio_of_word=Y_IOB[essay_id][word_id]

    if not (bio_of_word=='O'):
      argumentative=True

    # Add to the sentence every word that is not a simple \n or the end of the sentence (".").
    if not ( word=="ù" or word=="." ):
      sentence+=word+" "
      if word not in ignore_tokens:
        sentence_length+=1
    if word=="ù":
      num_paragraph+=1
      num_in_paragraph=0
          
    # . is the end of the sentence.
    if word in [".","?","!"] and ( not sentence=="") :
      sentence_list.append(sentence)

      # Add the indicator of the presence of argumentative sections to the correspondent lists.
      sentences_argumentative_map.append(argumentative)
      # Add the indicators of the sentence's position to the correspondent lists.
      number_of_paragraph.append(num_paragraph)
      number_sentence_in_paragraph.append(num_in_paragraph)
      number_sentence_in_the_essay.append(num_in_essay)
      sentence_essay_id_list.append(essay_id)
      # Strore the sentence length and set the value for the next sentence to 0.
      sentence_number_words.append(sentence_length)
      
      sentence_length=0
      argumentative=False
      sentence=""
      num_in_paragraph+=1
      num_in_essay+=1

  if not sentence=="":  
    sentence_list.append(sentence)

    # Add the indicator of the presence of argumentative sections to the correspondent lists.
    sentences_argumentative_map.append(argumentative)
    # Add the indicators of the sentence's position to the correspondent lists.
    number_of_paragraph.append(num_paragraph)
    number_sentence_in_paragraph.append(num_in_paragraph)
    number_sentence_in_the_essay.append(num_in_essay)
    sentence_essay_id_list.append(essay_id)
    # Strore the sentence length and set the value for the next sentence to 0.
    sentence_number_words.append(sentence_length)
      
    sentence_length=0
    argumentative=False
    sentence=""
    num_in_paragraph+=1
    num_in_essay+=1     

In [ ]:
len(sentence_list)

44936

In [ ]:
# Store for each sentence the length with respect to the media of the essay. 

sentence_length_with_respect_to_media_essay=[]

lengths_sentences=[]
total_sentence_length=0
counter_sentences=0

for i in range(len(sentence_number_words)):
  # Check if new essay.
  if number_sentence_in_the_essay[i]==0 and (not (counter_sentences==0)):
    media=total_sentence_length/counter_sentences
    for text in lengths_sentences:
      sentence_length_with_respect_to_media_essay.append(text/media)
    lengths_sentences=[]
    total_sentence_length=0
    counter_sentences=0
  sentence=sentence_number_words[i]
  lengths_sentences.append(sentence)
  total_sentence_length+=sentence
  counter_sentences+=1
  # check if sententce is the last one of the corpus.
  if i==len(number_sentence_in_the_essay)-1:
    media=total_sentence_length/counter_sentences
    for text in lengths_sentences:
      sentence_length_with_respect_to_media_essay.append(text/media)

In [ ]:
len(sentence_length_with_respect_to_media_essay)

44936

In [ ]:
print(number_of_paragraph[0:20])

[1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 8, 8, 8, 9]


In [ ]:
# Create a list containing, for each sentences in the corpus, the number of sentences in
# its own paragraph. 
count_sentences_in_paragraph=[]

len_sentence_list=len(sentence_list)-1

for i in range(len_sentence_list):
  if not number_of_paragraph[i]==number_of_paragraph[i+1]:
    number_to_add=number_sentence_in_paragraph[i]+1
    count_sentences_in_paragraph+=[number_to_add for j in range(number_to_add)]
  if i==(len_sentence_list-1):
    number_to_add=number_sentence_in_paragraph[i+1]+1
    count_sentences_in_paragraph+=[number_to_add for j in range(number_to_add)]

In [ ]:
count_sentences_in_paragraph[:10]

[1, 5, 5, 5, 5, 5, 2, 2, 3, 3]

In [ ]:
# Create a list containing, for each document in the corpus, the number of pargraphs
# in it. 
count_paragraphs_in_essay=[]

len_sentence_list=len(sentence_list)-1

for i in range(len_sentence_list):
  if not sentence_essay_id_list[i]==sentence_essay_id_list[i+1]:
    number_to_add=number_of_paragraph[i]+1
    number_of_addition=number_sentence_in_the_essay[i]+1
    count_paragraphs_in_essay+=[number_to_add for j in range(number_of_addition)]
  if i==(len_sentence_list-1):
    number_to_add=number_of_paragraph[i+1]+1
    number_of_addition=number_sentence_in_the_essay[i+1]+1
    count_paragraphs_in_essay+=[number_to_add for j in range(number_of_addition)]

In [ ]:
count_paragraphs_in_essay[:10]

[86, 86, 86, 86, 86, 86, 86, 86, 86, 86]

In [ ]:
for i in range(5):
  print(sentence_list[i])
  print("paragraph {} - position in paragraph {} - position in text {} - total number of sentences in current paragraph {}".format(number_of_paragraph[i],number_sentence_in_paragraph[i], number_sentence_in_the_essay[i], count_sentences_in_paragraph[i]))
  print("absolute sentence length {} - with respect to the media of the essay {}".format(sentence_number_words[i],sentence_length_with_respect_to_media_essay[i]))
  if sentences_argumentative_map[i]:
    print("argumentative")
  else:
    print("NOT argumentative")
  print("-")  

controversies over video games often center on topics such as video game graphic violence , sex and sexism , violent and gory scenes , partial or full nudity , portrayal of criminal behavior , racism , and other provocative and objectionable material 
paragraph 1 - position in paragraph 0 - position in text 0 - total number of sentences in current paragraph 1
absolute sentence length 36 - with respect to the media of the essay 1.3528399311531842
NOT argumentative
-
video games have been studied for links to addiction and aggression 
paragraph 2 - position in paragraph 0 - position in text 1 - total number of sentences in current paragraph 5
absolute sentence length 11 - with respect to the media of the essay 0.4133677567412507
NOT argumentative
-
earlier metaanalyses ( an analysis of several studies ) were conflicting 
paragraph 2 - position in paragraph 1 - position in text 2 - total number of sentences in current paragraph 5
absolute sentence length 9 - with respect to the media of t

In [ ]:
# Create dataframe of the feature extracted until now and export it.

sentence_stat_dataframe=pd.DataFrame(dict(zip(["paragraph in essay","sentence in paragraph", "sentence in essay", "total sentences in paragraph", "total paragraphs in document","number words in sentence", "sentence length / media sentence length in essay"],
                                              [number_of_paragraph,number_sentence_in_paragraph,number_sentence_in_the_essay,count_sentences_in_paragraph, count_paragraphs_in_essay,sentence_number_words,sentence_length_with_respect_to_media_essay])))

sentence_stat_dataframe.to_csv('Debater_sentence_statistic.csv',index=False)


# **Find out percentage of sentences which contains at least one argumentative section**

In [ ]:
# Number of total sententences contained in the essays.
num_total_sentences=len(sentence_list)
print("Total number of sentences {}".format(num_total_sentences))

# Number of sententences containing at least a labeled argumentative section.
num_arg_sentences=sum(sentences_argumentative_map)
print("Number of sentences containing an argumentative section {}".format(num_arg_sentences))

Total number of sentences 44936
Number of sentences containing an argumentative section 3777


# **Export sentences' labels**

In [ ]:
pd.DataFrame(sentences_argumentative_map).to_csv('Debater_argumentative_labels.csv',index=False)

# **POS preparation**

In [ ]:
# Separate each sentence in the list of POS TAG that compones them.
POS_sentence_list=[nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentence_list]

POS_sentence_list[0]

[('controversies', 'NNS'),
 ('over', 'IN'),
 ('video', 'JJ'),
 ('games', 'NNS'),
 ('often', 'RB'),
 ('center', 'VBP'),
 ('on', 'IN'),
 ('topics', 'NNS'),
 ('such', 'JJ'),
 ('as', 'IN'),
 ('video', 'JJ'),
 ('game', 'NN'),
 ('graphic', 'JJ'),
 ('violence', 'NN'),
 (',', ','),
 ('sex', 'NN'),
 ('and', 'CC'),
 ('sexism', 'NN'),
 (',', ','),
 ('violent', 'NN'),
 ('and', 'CC'),
 ('gory', 'NN'),
 ('scenes', 'NNS'),
 (',', ','),
 ('partial', 'JJ'),
 ('or', 'CC'),
 ('full', 'JJ'),
 ('nudity', 'NN'),
 (',', ','),
 ('portrayal', 'NN'),
 ('of', 'IN'),
 ('criminal', 'JJ'),
 ('behavior', 'NN'),
 (',', ','),
 ('racism', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('other', 'JJ'),
 ('provocative', 'JJ'),
 ('and', 'CC'),
 ('objectionable', 'JJ'),
 ('material', 'NN')]

In [ ]:
from nltk.tag import pos_tag, map_tag

# Create a version with the simplifyed version of the POS tag.

simplified_POS_sentence_list=[]

for POS_sentence in POS_sentence_list:
  simplified_POS_sentence_list.append([(word, map_tag('en-ptb', 'universal', tag)) for word, tag in POS_sentence])

In [ ]:
simplified_POS_sentence_list[0]

[('controversies', 'NOUN'),
 ('over', 'ADP'),
 ('video', 'ADJ'),
 ('games', 'NOUN'),
 ('often', 'ADV'),
 ('center', 'VERB'),
 ('on', 'ADP'),
 ('topics', 'NOUN'),
 ('such', 'ADJ'),
 ('as', 'ADP'),
 ('video', 'ADJ'),
 ('game', 'NOUN'),
 ('graphic', 'ADJ'),
 ('violence', 'NOUN'),
 (',', '.'),
 ('sex', 'NOUN'),
 ('and', 'CONJ'),
 ('sexism', 'NOUN'),
 (',', '.'),
 ('violent', 'NOUN'),
 ('and', 'CONJ'),
 ('gory', 'NOUN'),
 ('scenes', 'NOUN'),
 (',', '.'),
 ('partial', 'ADJ'),
 ('or', 'CONJ'),
 ('full', 'ADJ'),
 ('nudity', 'NOUN'),
 (',', '.'),
 ('portrayal', 'NOUN'),
 ('of', 'ADP'),
 ('criminal', 'ADJ'),
 ('behavior', 'NOUN'),
 (',', '.'),
 ('racism', 'NOUN'),
 (',', '.'),
 ('and', 'CONJ'),
 ('other', 'ADJ'),
 ('provocative', 'ADJ'),
 ('and', 'CONJ'),
 ('objectionable', 'ADJ'),
 ('material', 'NOUN')]

In [ ]:
# Get the list of the sentences as POS tag from the list of the
# list of the POS tag of each sentence. 
def get_POS_sentence_list(POS_sentence_list: list)->list:
  POS_sentences=[]
  for POS_list in POS_sentence_list:
    sentence=""
    for word, tag in POS_list:
      sentence+=tag+" "
    POS_sentences.append(sentence)    

  return POS_sentences

In [ ]:
POS_sentences=get_POS_sentence_list(POS_sentence_list)
simple_POS_sentences=get_POS_sentence_list(simplified_POS_sentence_list)

POS_sentences[0]

'NNS IN JJ NNS RB VBP IN NNS JJ IN JJ NN JJ NN , NN CC NN , NN CC NN NNS , JJ CC JJ NN , NN IN JJ NN , NN , CC JJ JJ CC JJ NN '

# **Customised Tokenizer**

In [ ]:
#  Lemmatization is used.
def build_tokenizer(text):

  wordnet_lemmatizer = WordNetLemmatizer()

  # Text tokenization.
  tokens=nltk.word_tokenize(text)

  # Text normalization throgh lemmatizzation.

  lemmas=[]
  for word in tokens:
    lemmas.append(wordnet_lemmatizer.lemmatize(word))

  return lemmas  

# **Bag of words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Trasformation of the passed corpus in the dataframe of the bag of ngram contained
# in it.
def bag_of_ngram(sentence_list: list, ngram: int)-> pd.DataFrame:
  vectorizer=CountVectorizer(tokenizer=build_tokenizer,ngram_range=(ngram,ngram))
  bag_ngram=vectorizer.fit_transform(sentence_list)

  dataframe=pd.DataFrame(bag_ngram.toarray(), columns=vectorizer.get_feature_names())

  return dataframe

# **POS bag of words**

In [ ]:
POS_BOW=bag_of_ngram(POS_sentences,1)
simple_POS_BOW=bag_of_ngram(simple_POS_sentences,1)

POS_BOW.head()

,#,$,'',(,),",",.,:,``,cc,cd,dt,ex,fw,in,jj,jjr,jjs,l,md,nn,nnp,nnps,nns,pdt,po,prp,rb,rbr,rp,sym,to,uh,vb,vbd,vbg,vbn,vbp,vbz,wdt,wp,wrb
0,0,0,0,0,0,6,0,0,0,5,0,0,0,0,4,10,0,0,0,0,11,0,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,3,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,2,1,0,0,0,0
2,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,1,4,0,0,6,2,0,1,0,0,8,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,2,0,0,0
4,0,0,0,1,1,0,0,0,0,1,0,2,0,0,3,1,0,0,0,0,3,0,0,2,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0


In [ ]:
# Print the number of occurrences not present in the BOW of the POS of the original 
# dataset (manually checked).
print(POS_BOW['#'].sum())
print(POS_BOW['nnps'].sum())
print(POS_BOW['l'].sum())

124
26
4


In [ ]:
# The sentences are separated by the character "-" and the last of 
# these characters is not followed by any values.
def get_values_from_file_imported(text_file)->list:
  text=open(text_file).read()

  values=text.split('-')

  values=values[:-1]

  return values

In [ ]:
imported_POS=get_values_from_file_imported('original_POS.txt')
imported_simple_POS=get_values_from_file_imported('original_simple_POS.txt')

In [ ]:
# Export the datasets.
POS_BOW=POS_BOW[imported_POS]
simple_POS_BOW=simple_POS_BOW[imported_simple_POS]

POS_BOW.to_csv('Debater_sentence_POS.csv',index=False)
simple_POS_BOW.to_csv('Debater_sentence_simple_POS.csv',index=False)

# **POS Bigrams**

In [ ]:
# Get the BOW of the bigrams of the POS tag.

POS_bigrams=bag_of_ngram(POS_sentences,2)
simple_POS_bigrams=bag_of_ngram(simple_POS_sentences,2)

POS_bigrams.head()

,# cd,# nn,$ #,$ $,$ '',$ (,$ ),"$ ,",$ .,$ :,$ ``,$ cc,$ cd,$ dt,$ fw,$ in,$ jj,$ jjr,$ jjs,$ md,$ nn,$ nnp,$ nns,$ po,$ rb,$ rbr,$ to,$ vbd,$ vbg,$ vbn,$ vbp,$ vbz,'' #,'' '','' (,'' ),"'' ,",'' .,'' :,'' ``,...,wp pdt,wp po,wp prp,wp rb,wp to,wp vb,wp vbd,wp vbg,wp vbn,wp vbp,wp vbz,wp wrb,wrb #,wrb (,wrb ),"wrb ,",wrb .,wrb ``,wrb cc,wrb cd,wrb dt,wrb ex,wrb in,wrb jj,wrb jjr,wrb jjs,wrb md,wrb nn,wrb nns,wrb pdt,wrb prp,wrb rb,wrb rbr,wrb to,wrb vb,wrb vbd,wrb vbg,wrb vbn,wrb vbp,wrb vbz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
imported_bigram_POS=get_values_from_file_imported('50_bigram_POS.txt')
imported_bigram_simple_POS=get_values_from_file_imported('50_bigram_simple_POS.txt')

In [ ]:
# They have dimension 50.
print(len(imported_bigram_POS))
print(len(imported_bigram_simple_POS))

50
50


In [ ]:
# Export the bag of bigrams of the POS tag for the 30 bigrams most related to 
# the argumentative label.

POS_bigrams=POS_bigrams[imported_bigram_POS]

POS_bigrams.to_csv('Debater_sentence_bigram_POS.csv',index=False)

In [ ]:
# Export the bag of bigrams of the POS tag for the list of 50 bigrams imported.

simple_POS_bigrams=simple_POS_bigrams[imported_bigram_simple_POS]

simple_POS_bigrams.to_csv('Debater_sentence_bigram_simple_POS.csv',index=False)

# **Dependencies**

In [ ]:
# Load the spacy engine:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

In [ ]:
# Return the list of dependencies for each sentence in the list passed in input
# and the maximum number of child and max depth of the ROOT(s) contained in the sentence.

def sentence_dependencies(sentence_list: list)->(list,int,int):
  
  sentence_dependency_list=[]
  max_child_ROOT=[]
  max_depth_ROOT=[]

  for sentence in sentence_list:
    doc = nlp(sentence)
    dependency_list=[]
    max_number_of_ROOT_child=0
    max_depth_of_tree=0
    for token in doc:
      dependency_list.append(token.dep_)
      if token.dep_=='ROOT':
        number_child=len([child for child in token.children])
        max_depth=walk_tree(token, 0)
        if number_child > max_number_of_ROOT_child:
          max_number_of_ROOT_child = number_child
        if max_depth >  max_depth_of_tree:
          max_depth_of_tree = max_depth
    max_child_ROOT.append(max_number_of_ROOT_child)
    max_depth_ROOT.append(max_depth_of_tree)
    sentence_dependency_list.append(dependency_list)   

  return sentence_dependency_list,max_child_ROOT, max_depth_ROOT

In [ ]:
sentence_dependency_list,max_child_ROOT, max_depth_ROOT = sentence_dependencies(sentence_list)

In [ ]:
sentence_dependency_list[0]

['nsubj',
 'prep',
 'compound',
 'pobj',
 'advmod',
 'ROOT',
 'prep',
 'pobj',
 'amod',
 'prep',
 'compound',
 'compound',
 'amod',
 'pobj',
 'punct',
 'conj',
 'cc',
 'conj',
 'punct',
 'amod',
 'cc',
 'conj',
 'conj',
 'punct',
 'amod',
 'cc',
 'conj',
 'conj',
 'punct',
 'conj',
 'prep',
 'amod',
 'pobj',
 'punct',
 'conj',
 'punct',
 'cc',
 'amod',
 'amod',
 'cc',
 'conj',
 'conj']

In [ ]:
max_child_ROOT[0:10]

[3, 4, 2, 2, 3, 2, 3, 3, 4, 2]

In [ ]:
max_depth_ROOT[0:10]

[13, 5, 5, 7, 6, 8, 2, 5, 11, 6]

In [ ]:
# Get the list of the sentences as dependency tag concatenating the tag 
# of the words in each sentence. 
def get_dependency_sentence_list(dep_sentence_list: list)->list:
  dep_sentences=[]
  for dep_list in dep_sentence_list:
    sentence=""
    for tag in dep_list:
      sentence+=tag+" "
    dep_sentences.append(sentence)    

  return dep_sentences

In [ ]:
dependency_sentences=get_dependency_sentence_list(sentence_dependency_list)

In [ ]:
dependency_sentences[0]

'nsubj prep compound pobj advmod ROOT prep pobj amod prep compound compound amod pobj punct conj cc conj punct amod cc conj conj punct amod cc conj conj punct conj prep amod pobj punct conj punct cc amod amod cc conj conj '

In [ ]:
dependency_BOW=bag_of_ngram(dependency_sentences,1)

dependency_BOW.head()

,acl,acomp,advcl,advmod,agent,amod,appos,attr,aux,auxpass,case,cc,ccomp,compound,conj,csubj,csubjpass,dative,dep,det,dobj,expl,intj,mark,meta,neg,nmod,npadvmod,nsubj,nsubjpass,nummod,oprd,parataxis,pcomp,po,pobj,preconj,predet,prep,prt,punct,quantmod,relcl,root,xcomp
0,0,0,0,1,0,7,0,0,0,0,0,5,0,3,10,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,0,0,4,0,6,0,0,1,0
1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,2,0,0,0,0,1,0
2,0,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,2,0,0,1,0
3,0,0,0,2,0,3,0,0,0,0,0,1,1,1,1,0,0,0,0,4,1,0,0,2,0,0,0,0,3,0,1,0,0,0,0,4,0,0,4,0,0,0,0,1,0
4,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,3,0,2,0,0,1,0


In [ ]:
# Number of occurrences of the row not present in the original dataset (manually checked).
dependency_BOW['meta'].sum()

149

In [ ]:
imported_dep=get_values_from_file_imported('dependency_tag.txt')

In [ ]:
# Export the bag of words of the 50 dependency tag most related to 
# the argumentative label.

dependency_BOW=dependency_BOW[imported_dep]

dependency_BOW.to_csv('Debater_sentence_dependency.csv',index=False)

In [ ]:
# Create dataframe of the feature regarding the dependency tree dimension.

dep_tree_stat_dataframe=pd.DataFrame(dict(zip(["tree depth","max number of children of a ROOT node"],[max_depth_ROOT, max_child_ROOT])))

In [ ]:
dep_tree_stat_dataframe.head()

,tree depth,max number of children of a ROOT node
0,13,3
1,5,4
2,5,2
3,7,2
4,6,3


In [ ]:
# Export the dataset of the dependency tree statistic.
dep_tree_stat_dataframe.to_csv('Debater_sentence_dependency_tree_stat.csv',index=False)

# **Named entities labels**

In [ ]:
# Return the list of strings containing the sequence of the labels of named 
# entities for each sentence in the list passed in input.

def sentence_named_entities(sentence_list: list)->list:
  sentence_entity_list=[]

  for sentence in sentence_list:
    doc = nlp(sentence)
    entity_sequence=''
    for token in doc.ents:
        entity_sequence+=token.label_+" "
    sentence_entity_list.append(entity_sequence)

  return sentence_entity_list

In [ ]:
# Return the list of the BOW of common labeled entity for each couple of 
# sentences in a possible relation.

label_named_entity_sentence_list=sentence_named_entities(sentence_list)

label_named_entity_BOW=bag_of_ngram(label_named_entity_sentence_list,1)

In [ ]:
label_named_entity_BOW.head()

,cardinal,date,event,fac,gpe,language,law,loc,money,norp,ordinal,org,percent,person,product,quantity,time,work_of_art
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
imported_lab_nam_ent=get_values_from_file_imported('labeled_named_entity.txt')

In [ ]:
# Export the dataset.
label_named_entity_BOW=label_named_entity_BOW[imported_lab_nam_ent]

label_named_entity_BOW.to_csv('Debater_sentence_named_entity_BOW.csv',index=False)

# **Type Token Ratio**

In [ ]:
from collections import Counter

type_token_ratio_list=[]

for sentence in sentence_list:
  word_seq=nltk.word_tokenize(sentence)

  types=Counter(word_seq)

  ttr=(len(types)/len(word_seq))*100
  
  type_token_ratio_list.append(ttr)

In [ ]:
print(type_token_ratio_list[0:10])

[78.57142857142857, 100.0, 100.0, 82.75862068965517, 100.0, 90.9090909090909, 100.0, 91.66666666666666, 92.10526315789474, 91.8918918918919]


# **Subjectivity**

In [ ]:
# Read the file
scores_text=open("subj_obj_Debater.txt").read()

In [ ]:
# Scores of the sentences are separated by the characters "--" and the last of 
# these characters is not followed by any score.
subjectivity_scores=scores_text.split('--')

subjectivity_scores=subjectivity_scores[:-1]

In [ ]:
subjectivity_scores[:10]

['0.00049',
 '0.98715',
 '0.98454',
 '0.99642',
 '0.99768',
 '0.05026',
 '0.00503',
 '2e-05',
 '1e-05',
 '0.00076']

In [ ]:
len(subjectivity_scores)

44936

# **Export subjectivity, token-ratio and named entity stats**

In [ ]:
# Create and export a dataframe containing the data extracted about the named entity
# in the corpus.

remeaning_extracted_data_dataframe=pd.DataFrame(dict(zip(["type token ratio","subjectivity score"],[ type_token_ratio_list,subjectivity_scores])))

remeaning_extracted_data_dataframe.to_csv('Debater_remaining_stat.csv',index=False)